In [ ]:
!pip install --upgrade openai python-dotenv networkx matplotlib

In [ ]:
from dotenv import load_dotenv
import os

os.environ.pop("OPENAI_API_KEY", None)

load_dotenv("/content/.env", override=True)

api_key = os.getenv("OPENAI_API_KEY")

print(api_key[:10] if api_key else "No key found")


sk-proj-ml


In [ ]:
image_path = "/content/LIC_LZ.png"

In [ ]:
from openai import OpenAI
import base64
import json
import re
import os

client = OpenAI(api_key=api_key)

# ==============================
# READ IMAGE
# ==============================
with open(image_path, "rb") as f:
    image_b64 = base64.b64encode(f.read()).decode()

# Detect MIME type
extension = os.path.splitext(image_path)[1].lower()
mime_map = {
    ".png": "image/png",
    ".jpg": "image/jpeg",
    ".jpeg": "image/jpeg",
    ".webp": "image/webp"
}
mime_type = mime_map.get(extension, "image/png")

prompt = """

YouAre a DISTINGUISHED PRINCIPAL ENTERPRISE CLOUD ARCHITECT performing a FULL RAW ARCHITECTURE INTELLIGENCE EXTRACTION.

Your mission is to convert this architecture diagram into the MOST EXHAUSTIVE POSSIBLE architecture knowledge graph.

THIS IS MAXIMUM RECALL MODE — ZERO LOSS EXTRACTION.

DO NOT filter, validate, deduplicate, optimize, rank, or suppress results.

Extract EVERYTHING visible or inferable.

I will review manually.

-----------------------------------------
GLOBAL EXTRACTION RULES
-----------------------------------------

• Include ALL elements
• Include ALL relationships
• Include explicit AND implicit relationships
• Include strong AND weak relationships
• Include uncertain relationships
• Include duplicates if unsure
• Include relationships without arrows
• Include relationships implied by placement
• Include relationships implied by architecture patterns
• Include control-plane dependencies
• Include data-plane dependencies
• Include cross-cutting services
• Include relationships even if direction is unclear
• Include inferred best-practice relationships

DO NOT OMIT ANYTHING.

-----------------------------------------
STEP 1 — ELEMENT EXTRACTION (MAX DETAIL)
-----------------------------------------

Identify EVERY element in the diagram, including:

Organizations / Tenants
Subscriptions / Accounts
Landing Zones
Management Groups
Regions / Availability Zones (only if shown)
VNets / VPCs
Subnets
Route tables
Network security groups
Firewalls
Load balancers
Application gateways
WAF
DDoS protections
Private endpoints
DNS services
Gateways
Peering connections
Compute services
Containers
Serverless services
Databases
Storage
Caches
AI/ML services
Integration services
API gateways
Identity services
Security services
Monitoring services
Governance services
DevOps tools
Management-plane services
Control-plane services
External systems
On-premises systems
Internet
Users
Unknown icons
Logical groupings
Service groups

For EACH element provide MAXIMUM DETAIL:

- Purpose
- Functional role
- What it hosts
- What depends on it
- What it depends on
- Control-plane vs data-plane
- Security role
- Networking role
- Data role
- Operational role
- Stateful/stateless if inferable
- Trust boundary role
- Exposure level (public/private)

If unsure, include best interpretation.

-----------------------------------------
STEP 2 — RELATIONSHIP EXTRACTION (EXHAUSTIVE)
-----------------------------------------

For EVERY pair of elements, include ALL possible relationships:

✔ Explicit (arrows)
✔ Implicit
✔ Inferred
✔ Best-practice assumptions
✔ Weak or uncertain
✔ Indirect dependencies
✔ Cross-layer relationships
✔ Control-plane relationships
✔ Management relationships

-----------------------------------------
STEP 3 — RELATIONSHIP DIMENSIONS (ALL)
-----------------------------------------

Capture relationships across ALL dimensions:

CONTAINMENT
COMPOSITION
CONNECTIVITY
NETWORK_FLOW (ingress, egress, routing, peering)
SECURITY
IDENTITY
DATA_FLOW
MONITORING
GOVERNANCE
DEPENDENCY
DEPLOYMENT
EXECUTION
MANAGEMENT
ACCESS_CONTROL
RESILIENCE
SCALING
BACKUP
FAILOVER
CONFIGURATION
OBSERVABILITY
EXPOSURE
TRUST_BOUNDARY

-----------------------------------------
STEP 4 — NETWORK & TRAFFIC ANALYSIS
-----------------------------------------

Include ALL flows:

Internet ingress paths
Egress paths
East-west traffic
Inspection paths
Routing paths
Trust boundaries
Segmentation
Public exposure
Private connectivity
Peering relationships

-----------------------------------------
STEP 5 — DATA & AI FLOW ANALYSIS
-----------------------------------------

Capture:

Read/write relationships
Data pipelines
AI inference flows
Training flows
Storage dependencies
Cache usage
Event flows
Integration flows

-----------------------------------------
STEP 6 — SECURITY & GOVERNANCE RELATIONSHIPS
-----------------------------------------

Include:

Defense layers
Security protections
Identity enforcement
Secrets management
Policy enforcement
Compliance controls
Threat monitoring
Risk boundaries

-----------------------------------------
STEP 7 — CONTROL-PLANE RELATIONSHIPS
-----------------------------------------

Include relationships even without arrows:

Monitoring → Observes → Resources
Identity → Authenticates → Services
Governance → Applies → Subscriptions
Management → Configures → Services
Policies → Enforce → Resources

-----------------------------------------
STEP 8 — IMPLICIT ARCHITECTURAL DEPENDENCIES
-----------------------------------------

Infer dependencies such as:

Service → Uses → Network
Service → Uses → DNS
Service → Uses → Identity
Service → Logs → Monitoring
Service → Secured_by → Security controls
Service → Managed_by → Governance
Service → Deploys_to → Compute

-----------------------------------------
STEP 9 — HARD RELATIONSHIP EXPANSION ENFORCEMENT
-----------------------------------------

MANDATORY COMPLETENESS RULE:

After containment relationships are extracted,
you MUST generate relationships across ALL other dimensions
for EVERY element using architectural reasoning.

Containment-only output is considered INCOMPLETE.

You MUST expand relationships across:

Connectivity
Network flows
Security protections
Identity interactions
Monitoring coverage
Data dependencies
Deployment relationships
Management relationships
Access paths
External integrations

For EACH element, reason:

What connects to it?
What secures it?
What monitors it?
What authenticates it?
What data flows through it?
What depends on it?
What does it depend on?
What manages it?

Generate relationships accordingly.

If output is dominated by CONTAINMENT relationships,
continue generating relationships
until cross-dimensional coverage is achieved.

-----------------------------------------
CRITICAL RULES
-----------------------------------------

• DO NOT validate correctness
• DO NOT remove duplicates
• DO NOT minimize output
• DO NOT prioritize confidence
• DO NOT skip uncertain items
• DO NOT enforce completeness checks
• DO NOT stop early
• Your response MUST be ONLY a JSON object. No other text, no apologies, no markdown outside the JSON.
• If you cannot extract any information, return an empty JSON object like `{"elements": [], "relationships": []}`.
• If you encounter any internal limitations or inability to process, embed that message within a 'processing_status' field in the JSON, but still return valid JSON.

Produce the MOST EXHAUSTIVE RESULT POSSIBLE.

-----------------------------------------
OUTPUT FORMAT — STRICT JSON
-----------------------------------------

{
  "elements": [
    {
      "name": "",
      "type": "",
      "cloud": "",
      "container": "",
      "description": "",
      "roles": [],
      "notes": ""
    }
  ],
  "relationships": [
    {
      "source": "",
      "target": "",
      "relationship_type": "",
      "dimension": "",
      "confidence": "High|Medium|Low",
      "explicit_arrow": true,
      "notes": ""
    }
  ]
}

OUTPUT ONLY JSON.
"""

# ==============================
# MULTIMODAL REQUEST
# ==============================
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{
        "role": "user",
        "content": [
            {"type": "text", "text": prompt},
            {
                "type": "image_url",
                "image_url": { "url": f"data:{mime_type};base64,{image_b64}" } # Corrected format
            }
        ]
    }],
    max_tokens=8000
)

raw_content = response.choices[0].message.content

# ==============================
# SAFE JSON EXTRACTION
# ==============================
json_match = re.search(r'\{[\s\S]*\}', raw_content)

if not json_match:
    print(raw_content)
    raise ValueError("No JSON found in model output")

json_string = json_match.group(0)

# Remove comments if present
json_string = re.sub(r'//.*', '', json_string)

try:
    data = json.loads(json_string)
except json.JSONDecodeError as e:
    print(f"JSON Decode Error: {e}")
    print(f"Attempting to repair JSON string...")

    # A common issue is missing commas between elements or objects.
    # This is a very simple attempt to 'heal' - might not cover all cases.
    # It tries to insert commas where an object/array ends immediately followed by another.
    repaired_json_string = re.sub(r'\}\s*\{', '},{', json_string)
    repaired_json_string = re.sub(r'\]\s*\[', '],[', repaired_json_string)

    try:
        data = json.loads(repaired_json_string)
        print("JSON successfully repaired and loaded.")
    except json.JSONDecodeError as e_repaired:
        print(f"JSON Repair failed. Original error: {e}. Repair attempt error: {e_repaired}")
        # Print problematic section for debugging
        lines = json_string.splitlines()
        error_line_number_match = re.search(r'line (\d+)', str(e))
        error_line_number = int(error_line_number_match.group(1)) if error_line_number_match else -1
        if error_line_number != -1 and 1 <= error_line_number <= len(lines):
            start_line = max(0, error_line_number - 5)
            end_line = min(len(lines), error_line_number + 5)
            print("\n--- Problematic JSON section ---")
            for i in range(start_line, end_line):
                print(f"{i + 1}: {lines[i]}")
            print("--------------------------------")
        raise e_repaired # Re-raise the error if repair fails

print("\n===== ARCHITECTURE EXTRACTION COMPLETE =====")
print(json.dumps(data, indent=2))


===== ARCHITECTURE EXTRACTION COMPLETE =====
{
  "elements": [
    {
      "name": "Azure AI Landing Zone",
      "type": "Landing Zone",
      "cloud": "Azure",
      "container": null,
      "description": "Base architecture for deploying AI services in Azure",
      "roles": [
        "Infrastructure setup",
        "AI Service Deployment"
      ],
      "notes": ""
    },
    {
      "name": "Ext Systems",
      "type": "External Systems",
      "cloud": null,
      "container": "Azure AI Landing Zone",
      "description": "External systems for integration",
      "roles": [
        "Integration"
      ],
      "notes": ""
    },
    {
      "name": "User",
      "type": "User",
      "cloud": null,
      "container": "Azure AI Landing Zone",
      "description": "End user accessing services",
      "roles": [
        "Consumer"
      ],
      "notes": ""
    },
    {
      "name": "Internet",
      "type": "External Network",
      "cloud": null,
      "container": "Azure AI Lan

In [ ]:
validation_prompt = """
You are a principal enterprise architect and cloud security reviewer performing an Architecture Intelligence Validation.

You are given a structured architecture model consisting of elements and relationships extracted from a diagram.

Your task is to validate completeness, correctness, and architectural soundness.

------------------------------------
VALIDATION OBJECTIVES
------------------------------------

1) RELATIONSHIP COMPLETENESS

Check whether relationships are missing for any element.

For EACH element, verify existence of:

- Containment relationship
- Connectivity relationships (if networked)
- Security protections
- Identity dependencies (if applicable)
- Monitoring coverage
- Governance controls
- Data dependencies (if applicable)

Flag elements lacking expected relationships.

------------------------------------
2) CONTAINMENT CONSISTENCY

Validate hierarchy:

Subscription → VNet → Subnet → Service

Detect:

- Services without subnet placement
- Subnets without parent VNet
- VNets without subscription
- Misplaced resources

------------------------------------
3) SECURITY COVERAGE ANALYSIS

Verify that critical services have protection:

Examples:

- Internet-facing services → WAF / Firewall
- VNets → NSGs / Firewall / DDoS
- Applications → Identity + Secrets management
- Data stores → Encryption + access control

Identify unprotected components.

------------------------------------
4) MONITORING & OPERATIONS COVERAGE

Check if components are monitored by:

- Network monitoring services
- Logging services
- Diagnostics services
- Security monitoring

Identify blind spots.

------------------------------------
5) IDENTITY & ACCESS VALIDATION

Ensure services that require authentication have identity providers.

Detect:

- Services lacking identity integration
- Missing role-based access controls
- Secrets without vault integration

------------------------------------
6) NETWORK ARCHITECTURE VALIDATION

Check:

- Ingress paths
- Egress paths
- Segmentation between zones
- DMZ isolation
- East-west traffic controls

Detect:

- Unrestricted connectivity
- Missing segmentation
- Potential lateral movement paths

------------------------------------
7) DATA FLOW VALIDATION

Verify:

- Data stores are accessed by appropriate services
- Sensitive data paths are protected
- Backup/replication relationships exist

------------------------------------
8) RESILIENCE & AVAILABILITY

Check for:

- Multi-AZ or multi-region design (if applicable)
- Single points of failure
- Lack of redundancy

------------------------------------
9) GOVERNANCE & COMPLIANCE

Verify:

- Policies applied
- Security governance coverage
- Compliance controls

------------------------------------
OUTPUT FORMAT — STRICT JSON
------------------------------------

{
  "validation_summary": {
    "overall_completeness_score": "",
    "critical_gaps_count": "",
    "high_risk_issues_count": ""
  },
  "missing_relationships": [
    {
      "element": "",
      "missing_relationship_type": "",
      "reason": ""
    }
  ],
  "security_gaps": [
    {
      "component": "",
      "issue": "",
      "risk_level": "Critical|High|Medium|Low"
    }
  ],
  "architecture_issues": [
    {
      "issue_type": "Containment|Network|Identity|Data|Monitoring|Governance",
      "description": ""
    }
  ],
  "single_points_of_failure": [
    {
      "component": "",
      "reason": ""
    }
  ],
  "recommended_relationships_to_add": [
    {
      "source": "",
      "target": "",
      "relationship_type": "",
      "dimension": ""
    }
  ]
}

------------------------------------
IMPORTANT RULES
------------------------------------

- Be extremely strict
- Assume enterprise-grade standards
- Do NOT fabricate relationships
- Identify risks and gaps clearly
- Output ONLY JSON
"""

# Corrected API call to use client.chat.completions.create
validation_response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{
        "role": "user",
        "content": [{"type": "text", "text": validation_prompt + "\n\nHere is the extracted architecture data to validate:\n" + json.dumps(data, indent=2)}]
    }],
    max_tokens=8000 # Increased max_tokens for a potentially large validation response
)

raw_content = validation_response.choices[0].message.content

# Extract JSON from markdown
json_start = raw_content.find('```json')
json_end = raw_content.rfind('```')

if json_start != -1 and json_end != -1 and json_start < json_end:
    json_string = raw_content[json_start + len('```json'):json_end].strip()
else:
    json_string = raw_content.strip() # Fallback if no markdown block is found

validation_data = json.loads(json_string)
print(validation_data)

{'validation_summary': {'overall_completeness_score': '70%', 'critical_gaps_count': '5', 'high_risk_issues_count': '3'}, 'missing_relationships': [{'element': 'Azure AI Landing Zone', 'missing_relationship_type': 'Connectivity', 'reason': 'The landing zone should have explicit connectivity paths defined for ingress and egress.'}, {'element': 'User', 'missing_relationship_type': 'Security protections', 'reason': 'User interactions with the system should be monitored for security compliance.'}, {'element': 'AI Services Virtual Network', 'missing_relationship_type': 'Governance controls', 'reason': 'Lacks explicit governance control relationships.'}, {'element': 'AI Services Virtual Network', 'missing_relationship_type': 'Monitoring coverage', 'reason': 'Missing comprehensive network monitoring integration.'}, {'element': 'Redis Cache', 'missing_relationship_type': 'Security protections', 'reason': 'Redis Cache must have encryption and access control measures defined.'}], 'security_gaps':

In [ ]:
import networkx as nx

G = nx.DiGraph()

# Add elements as nodes
for element in data["elements"]:
    G.add_node(element["name"], type=element["type"], cloud=element["cloud"])

# Add relationships as edges
for relationship in data["relationships"]:
    G.add_edge(
        relationship["source"],
        relationship["target"],
        relationship_type=relationship["relationship_type"],
        confidence=relationship["confidence"], # Use 'confidence' instead of 'dimension'
        explicit_arrow=relationship["explicit_arrow"],
        notes=relationship["notes"] # Use 'notes' instead of 'inferred' and 'description'
    )

print("Graph created with", len(G.nodes), "nodes and", len(G.edges), "edges")

Graph created with 71 nodes and 59 edges


In [ ]:
def graph_rag_query(question):

    # Find relevant components by keyword match
    relevant_nodes = [
        node for node in G.nodes
        if node.lower() in question.lower()
    ]

    context = []

    # If no direct match, include whole graph summary
    if not relevant_nodes:
        relevant_nodes = list(G.nodes)[:5]  # fallback sample

    for component in relevant_nodes:

        # Outgoing interactions
        for target in G.successors(component):
            relation = G[component][target]['relation']
            context.append(f"{component} --{relation}--> {target}")

        # Incoming interactions
        for source in G.predecessors(component):
            relation = G[source][component]['relation']
            context.append(f"{source} --{relation}--> {component}")

    context_text = "\n".join(context)

    prompt = f"""
    You are an enterprise Azure architecture assistant.

    Use ONLY the graph interactions below to answer.

    Graph Context:
    {context_text}

    Question:
    {question}
    """

    response = client.responses.create(
        model="gpt-4.1",
        input=prompt
    )

    return response.output_text


In [ ]:
from openai import OpenAI

client = OpenAI()

def graph_rag_query(question):

    # 🔎 Identify relevant components mentioned in the question
    relevant_nodes = [
        node for node in G.nodes
        if node.lower() in question.lower()
    ]

    context = []

    # Fallback if no direct match
    if not relevant_nodes:
        relevant_nodes = list(G.nodes)[:5]

    # 🔗 Build context safely
    for component in relevant_nodes:

        # Outgoing interactions
        for target in G.successors(component):
            # Use 'relationship_type' and 'dimension' as defined in graph creation
            relationship_type = G[component][target].get('relationship_type', 'INTERACTS_WITH')
            dimension = G[component][target].get('dimension', '')
            context.append(f"{component} --{relationship_type} ({dimension})--> {target}")

        # Incoming interactions
        for source in G.predecessors(component):
            # Use 'relationship_type' and 'dimension' as defined in graph creation
            relationship_type = G[source][component].get('relationship_type', 'INTERACTS_WITH')
            dimension = G[source][component].get('dimension', '')
            context.append(f"{source} --{relationship_type} ({dimension})--> {component}")

    context_text = "\n".join(context)

    prompt = f"""
    You are an enterprise Azure architecture assistant.

    Use ONLY the graph interactions below to answer.

    Graph Context:
    {context_text}

    Question:
    {question}
    """

    # Updated to use client.chat.completions.create with correct message structure
    response = client.chat.completions.create(
        model="gpt-4o", # Using gpt-4o for consistency
        messages=[{
            "role": "user",
            "content": [{"type": "text", "text": prompt}]
        }],
        max_tokens=1000 # Adjusted max_tokens for text response
    )

    return response.choices[0].message.content

In [ ]:
while True:
    query = input("\nAsk about the architecture (or 'exit'): ")

    if query.lower() == "exit":
        break

    answer = graph_rag_query(query)

    print("\n===== AI RESPONSE =====\n")
    print(answer)



Ask about the architecture (or 'exit'): give the network path of Azure AI Foundry

===== AI RESPONSE =====

Based on the graph interactions you have provided, it appears that there is no direct mention of "Azure AI Foundry" in the graph context. However, if we consider the typical network paths for services involving Azure and on-premises networks, a plausible path for a service running on Azure (like Azure AI Foundry) interacting with on-premise resources via the Internet and ExpressRoute could be:

1. Azure AI Foundry (assumed to be part of Ext Systems) -- uses "Integration" to connect to User.
2. User -- uses "Access" to connect to the Internet.
3. Internet -- uses "Access" to connect to the On-premises network.
4. On-premises network -- uses "Connectivity" via ExpressRoute.

So, the network path from Azure AI Foundry to an on-premises network might look like this:

Azure AI Foundry --> User --> Internet --> On-premises network --> ExpressRoute

Please note that without explicit me